# Pyspark 2: DataFrame

To start working with dataframes, we have to create **SparkSession** object from our SparkContext object. <br/>
We can think SparkContext object as a connection with cluster, SparkSession as our interface with the connection <br/> 
**SparkSession** is an entry point to dataframe API. 

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
import warnings

In [4]:
sc= SparkContext(appName="SSDM", master="local[*]").getOrCreate()

In [5]:
ss = SparkSession.builder.appName('SDDM_2').getOrCreate()
ss.version

'2.4.4'

**Creating Dataframe in spark  (There are 2 different methods.)**
- From existing RDDS by using SparkSession's createDataFrame() method <br/>
- From various data sources; CSV, JSON, TXT by using SparkSession read method. 

Schema controls the data and provides info about column name, type of data inthe column, empty values etc. 

In [16]:
#create dataframe from RDD. 
iphone_rdd= sc.parallelize([
    ("XS", 2018),
    ("XR", 2018),
    ("X10", 2017),
    ("8Plus", 2017)
])

names= ["model", "year"]

df= ss.createDataFrame(iphone_rdd, schema=names)
df.show()

+-----+----+
|model|year|
+-----+----+
|   XS|2018|
|   XR|2018|
|  X10|2017|
|8Plus|2017|
+-----+----+



In [17]:
df.printSchema()  #prints the type of columns

root
 |-- model: string (nullable = true)
 |-- year: long (nullable = true)



In [9]:
#creating dataframe by reading csv/json/txt file

df_csv= ss.read.csv("Life_Expectancy_Data.csv",
                      header=True, inferSchema=False)

df_csv.show(5)

+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+----------+---------------------+-------------------+-------------------------------+---------+
|    Country|Year|    Status|Life expectancy |Adult Mortality|infant deaths|Alcohol|percentage expenditure|Hepatitis B|Measles | BMI |under-five deaths |Polio|Total expenditure|Diphtheria | HIV/AIDS|       GDP|Population| thinness  1-19 years| thinness 5-9 years|Income composition of resources|Schooling|
+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+----------+---------------------+-------------------+-------------------------------+---------+
|Afghanistan|2015|Developing|              65|            263|           62|   0.0

In [26]:
df_csv.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Life expectancy : string (nullable = true)
 |-- Adult Mortality: string (nullable = true)
 |-- infant deaths: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- percentage expenditure: string (nullable = true)
 |-- Hepatitis B: string (nullable = true)
 |-- Measles : string (nullable = true)
 |--  BMI : string (nullable = true)
 |-- under-five deaths : string (nullable = true)
 |-- Polio: string (nullable = true)
 |-- Total expenditure: string (nullable = true)
 |-- Diphtheria : string (nullable = true)
 |--  HIV/AIDS: string (nullable = true)
 |-- GDP: string (nullable = true)
 |-- Population: string (nullable = true)
 |--  thinness  1-19 years: string (nullable = true)
 |--  thinness 5-9 years: string (nullable = true)
 |-- Income composition of resources: string (nullable = true)
 |-- Schooling: string (nullable = true)



#### DataFrame operations in Pysprak
DataFrame transformations: <br/>
- select() 
- filter()
- groupby()
- dropDuplicates()
- withColumnRenamed()

DataFRame Actions: <br/>
- printSchema()
- head()
- show()
- count()
- columns()
- describe()

In [4]:
! wget -q -O - https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2017-06.csv | head -n 5000 > tmp.csv


In [7]:
df= ss.read.csv("tmp.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- Dispatching_base_num: string (nullable = true)
 |-- Pickup_DateTime: timestamp (nullable = true)
 |-- DropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)



In [12]:
df= df_csv

In [13]:
df.show(5)

+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+----------+---------------------+-------------------+-------------------------------+---------+
|    Country|Year|    Status|Life expectancy |Adult Mortality|infant deaths|Alcohol|percentage expenditure|Hepatitis B|Measles | BMI |under-five deaths |Polio|Total expenditure|Diphtheria | HIV/AIDS|       GDP|Population| thinness  1-19 years| thinness 5-9 years|Income composition of resources|Schooling|
+-----------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+----------+----------+---------------------+-------------------+-------------------------------+---------+
|Afghanistan|2015|Developing|              65|            263|           62|   0.0

In [14]:
df.count()

2938

In [18]:
# select() trnasformation subsets the columns in the dataframe
df_year= df.select("Year")
df_year.show(5)

+----+
|Year|
+----+
|2015|
|2014|
|2013|
|2012|
|2011|
+----+
only showing top 5 rows



In [27]:
# filter transformation filters out the rows based on condition
df_filtered= df_year.filter(df_year["Year"]>2012)
df_filtered.show(5)

+----+
|Year|
+----+
|2015|
|2014|
|2013|
|2015|
|2014|
+----+
only showing top 5 rows



In [28]:
# we can also use square brackets like in pandas instead of filter()
df_year[df_year.Year>2012].show(5)

+----+
|Year|
+----+
|2015|
|2014|
|2013|
|2015|
|2014|
+----+
only showing top 5 rows



In [31]:
df[(df.Year>2012) & (df.Country=="Turkey")].show()

+-------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+-----------+----------+---------------------+-------------------+-------------------------------+---------+
|Country|Year|    Status|Life expectancy |Adult Mortality|infant deaths|Alcohol|percentage expenditure|Hepatitis B|Measles | BMI |under-five deaths |Polio|Total expenditure|Diphtheria | HIV/AIDS|        GDP|Population| thinness  1-19 years| thinness 5-9 years|Income composition of resources|Schooling|
+-------+----+----------+----------------+---------------+-------------+-------+----------------------+-----------+--------+-----+------------------+-----+-----------------+-----------+---------+-----------+----------+---------------------+-------------------+-------------------------------+---------+
| Turkey|2015|Developing|            75.8|             16|           15|   null|           

In [35]:
df= df.select("Country", "Year", "Life expectancy ")
df.show(5)

+-----------+----+----------------+
|    Country|Year|Life expectancy |
+-----------+----+----------------+
|Afghanistan|2015|              65|
|Afghanistan|2014|            59.9|
|Afghanistan|2013|            59.9|
|Afghanistan|2012|            59.5|
|Afghanistan|2011|            59.2|
+-----------+----+----------------+
only showing top 5 rows



In [37]:
# groupby() operation can be used to grouping variables. 
df_group= df.groupby("Country")
df_group.count().show()

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|       Côte d'Ivoire|   16|
|                Chad|   16|
|Micronesia (Feder...|   16|
|            Paraguay|   16|
|               Yemen|   16|
|             Senegal|   16|
|          Cabo Verde|   16|
|              Sweden|   16|
|            Kiribati|   16|
|   Republic of Korea|   16|
|              Guyana|   16|
|             Eritrea|   16|
|         Philippines|   16|
|            Djibouti|   16|
|               Tonga|   16|
|            Malaysia|   16|
|           Singapore|   16|
|                Fiji|   16|
|              Turkey|   16|
|              Malawi|   16|
+--------------------+-----+
only showing top 20 rows



In [38]:
# orderby() operations sorts the DataFrame based on or more columns
df_group.count().orderBy("Country").show()

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|         Afghanistan|   16|
|             Albania|   16|
|             Algeria|   16|
|              Angola|   16|
| Antigua and Barbuda|   16|
|           Argentina|   16|
|             Armenia|   16|
|           Australia|   16|
|             Austria|   16|
|          Azerbaijan|   16|
|             Bahamas|   16|
|             Bahrain|   16|
|          Bangladesh|   16|
|            Barbados|   16|
|             Belarus|   16|
|             Belgium|   16|
|              Belize|   16|
|               Benin|   16|
|              Bhutan|   16|
|Bolivia (Plurinat...|   16|
+--------------------+-----+
only showing top 20 rows



In [50]:
# withColumnRenamed renames the column in dataframe 
df= df.withColumnRenamed("Life expectancy ", "life_expectancy")
df.columns

['Country', 'Year', 'life_excpectancy', 'new_col']

In [54]:
# creating a new column from existing one
df= df.withColumn("new_col", df["Year"]/df["life_excpectancy"])
df= df.withColumnRenamed("life_excpectancy", "life_expectancy")
df.columns

['Country', 'Year', 'life_expectancy', 'new_col']

In [55]:
#droping a column
df= df.drop("country") #no axis concept

#### Pandafy Spark Dataframe

In [57]:
import pandas as pd
import numpy as np

In [59]:
#create pandas dataframe
pd_temp= pd.DataFrame(np.random.random(10))

# create spark df from pd_temp
spark_temp= ss.createDataFrame(pd_temp)
spark_temp.show()

+-------------------+
|                  0|
+-------------------+
| 0.6830589835151355|
|  0.938442031648775|
| 0.6620537182398036|
| 0.4810336711302572|
| 0.8248168948038054|
|0.29155925927581894|
|  0.380027726730344|
| 0.2863062436454975|
| 0.2773653857807624|
|0.04542075900012721|
+-------------------+



In [6]:
import numpy as np
import pandas as pd

In [7]:
import pyspark.sql.functions as F

In [8]:
sc.stop()

In [9]:
ss.stop()